![Clothing Classifier Model](Clothing%20Classifier%20Model.png)


Fashion Forward is a new AI-based e-commerce clothing retailer.
They want to use image classification to automatically categorize new product listings, making it easier for customers to find what they're looking for. It will also assist in inventory management by quickly sorting items.

As a data scientist tasked with implementing a garment classifier, your primary objective is to develop a machine learning model capable of accurately categorizing images of clothing items into distinct garment types such as shirts, trousers, shoes, etc.


In [97]:
# Run the cells below first

In [98]:
!pip install torchmetrics

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [99]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall

In [100]:
# Load datasets
from torchvision import datasets
import torchvision.transforms as transforms

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [101]:
# Number of classes
classes = train_data.classes
num_classes = len(train_data.classes)

In [102]:
num_input_channels = 1
num_output_channels = 16
image_size = train_data[0][0].shape[1]

In [103]:
# Train dataset
dataloader_train = DataLoader(
    train_data,
    shuffle=True,
    batch_size=10
)

image_train, label_train = next(iter(dataloader_train))
print(image_train.shape)

torch.Size([10, 1, 28, 28])


In [104]:
# 1. Define CNN

class MultiClassImageClassifier(nn.Module):
  
    # Define the init method
    def __init__(self, num_classes):
        super(MultiClassImageClassifier, self).__init__()
        self.conv1 = nn.Conv2d(num_input_channels, num_output_channels, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

        # Create a fully connected layer
        self.fc = nn.Linear(num_output_channels * (image_size//2)**2, num_classes)
        
    def forward(self, x):
        # Pass inputs through each layer
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

In [105]:
# Define training function
def train_model(optimizer, net, num_epochs):
    num_processed = 0
    criterion = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        running_loss = 0
        num_processed = 0
        for features, labels in dataloader_train:
            optimizer.zero_grad()
            outputs = net(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            num_processed += len(labels)
        print(f'epoch {epoch}, loss: {running_loss / num_processed}')
        
    train_loss = running_loss / len(dataloader_train)

In [106]:
# 2. Train for 1 epoch
net = MultiClassImageClassifier(num_classes)
optimizer = optim.Adam(net.parameters(), lr=0.001)

train_model(
    optimizer=optimizer,
    net=net,
    num_epochs=1,
)


epoch 0, loss: 0.04050624387549081


In [107]:
# 2. Train the CNN

#net = MultiClassImageClassifier(num_classes=10)

#criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(net.parameters(), lr = 0.001)

#for epoch in range(1):
#    for images, labels in dataloader_train:
#        optimizer.zero_grad()
#        outputs = net(images)
#        loss = criterion(outputs, labels)
#        loss.backward()
#        optimizer.step()

In [108]:
# Test Dataset

dataloader_test = DataLoader(
    test_data,
    shuffle=False,
    batch_size=10
)

image_test, label_test = next(iter(dataloader_test))
print(image_test.shape)

torch.Size([10, 1, 28, 28])


In [109]:
# 3. Test the CNN

from torchmetrics import Accuracy, Recall, Precision

metric_accuracy = Accuracy(task= 'multiclass', num_classes=num_classes)
metric_recall = Recall(task= 'multiclass', num_classes=num_classes, average=None)
metric_precision = Precision(task='multiclass', num_classes=num_classes, average=None)

# Run model on test set
net.eval()
predicted = []
for i, (features, labels) in enumerate(dataloader_test):
    output = net.forward(features.reshape(-1, 1, image_size, image_size))
    cat = torch.argmax(output, dim=-1)
    predicted.extend(cat.tolist())
    metric_accuracy(cat, labels)
    metric_precision(cat, labels)
    metric_recall(cat, labels)

# Compute the metrics
accuracy = metric_accuracy.compute().item()
precision = metric_precision.compute().tolist()
recall = metric_recall.compute().tolist()




In [110]:
# MEtrics:
print('Accuracy:', accuracy)
print('Recall:', recall)
print('Precision:', precision)

Accuracy: 0.8845999836921692
Recall: [0.8149999976158142, 0.9649999737739563, 0.8230000138282776, 0.9079999923706055, 0.8019999861717224, 0.9779999852180481, 0.6850000023841858, 0.9419999718666077, 0.9729999899864197, 0.9549999833106995]
Precision: [0.8350409865379333, 0.9887295365333557, 0.8164682388305664, 0.8631178736686707, 0.8242548704147339, 0.9616519212722778, 0.6822709441184998, 0.9515151381492615, 0.9643211364746094, 0.9597989916801453]


In [111]:
# Confusion Matrix

import torch
import numpy as np
from torchmetrics import ConfusionMatrix

#pred_torch = np.array(predictions, dtype=np.float32)
#label_torch = np.array(labelss, dtype=np.int64)

#confmat = ConfusionMatrix(task="multiclass", num_classes=10)
#confmat(torch.from_numpy(pred_torch), torch.from_numpy(label_torch))